In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
datadir = "/content/drive/MyDrive/cs411_proj"
os.chdir(datadir)
!pwd

/content/drive/MyDrive/cs411_proj


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df=pd.read_csv("reviews.csv")
print(df.head())

   Id                                             Review  Label
0   0                               good and interesting      5
1   1  This class is very helpful to me. Currently, I...      5
2   2  like!Prof and TAs are helpful and the discussi...      5
3   3  Easy to follow and includes a lot basic and im...      5
4   4  Really nice teacher!I could got the point eazl...      4


In [ ]:
df["Label"].nunique()

5

In [ ]:
df.loc[df["Label"].isin([1]),"Label"]=0

In [ ]:
df.loc[df["Label"].isin([2,3,4]),"Label"]=1
df.loc[df["Label"].isin([5]),"Label"]=2

In [ ]:
df["Label"].nunique()

3

In [ ]:
df.head()

,Id,Review,Label
0,0,good and interesting,2
1,1,"This class is very helpful to me. Currently, I...",2
2,2,like!Prof and TAs are helpful and the discussi...,2
3,3,Easy to follow and includes a lot basic and im...,2
4,4,Really nice teacher!I could got the point eazl...,1


In [ ]:
df['Label'].value_counts()

Label
2    79173
1    25376
0     2469
Name: count, dtype: int64

In [ ]:
samples_label_0 = df[df['Label'] == 0].sample(n=2469, random_state=42)
samples_label_1 = df[df['Label'] == 1].sample(n=5000, random_state=42)
samples_label_2 = df[df['Label'] == 2].sample(n=5000, random_state=42)
sampled_df = pd.concat([samples_label_0, samples_label_1, samples_label_2])

In [ ]:
X=sampled_df["Review"]
y=sampled_df["Label"]

In [ ]:
def tokenize_and_stem(text):
    stemmer = PorterStemmer()
    tokens = [word for word in word_tokenize(text.lower()) if word.isalpha()]
    filtered_tokens = [stemmer.stem(token) for token in tokens if token not in stopwords.words('english')]
    return filtered_tokens

tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenize_and_stem)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


rf_pipeline = Pipeline([
    ('tfidf', tfidf_vectorizer),
    ('clf', RandomForestClassifier(random_state=42))
])


svm_pipeline = Pipeline([
    ('tfidf', tfidf_vectorizer),
    ('clf', SVC(kernel='linear', probability=True, random_state=42))
])


# rf_pipeline.fit(X_train, y_train)
# y_pred_rf = rf_pipeline.predict(X_test)
# print('RandomForest Accuracy:', accuracy_score(y_test, y_pred_rf))


svm_pipeline.fit(X_train, y_train)
y_pred_svm = svm_pipeline.predict(X_test)
print('SVM Accuracy:', accuracy_score(y_test, y_pred_svm))


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


SVM Accuracy: 0.6683771648492624


In [ ]:
type(X_test)

pandas.core.series.Series

In [ ]:
text="Despite repeated warnings, fails to improve"
tst=pd.Series(text)
test_pred_rf=svm_pipeline.predict(tst)
print(test_pred_rf)

[0]


In [ ]:
from joblib import dump
dump(svm_pipeline, 'svm_pipeline.joblib')

['svm_pipeline.joblib']

In [ ]:
from joblib import load
test_model=load('svm_pipeline.joblib')

In [ ]:
new_text="Great performance"
tst=pd.Series(new_text)
ans=test_model.predict(tst)

In [ ]:
ans[0]

2